In [22]:
import sys
sys.path.insert(0, '../feature_generation')
sys.path.insert(0, '../..')
import pandas as pd
import numpy as np
import keras
import models
import paths
import shutil
from PIL import Image
import math
from os.path import join
from os import listdir
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# pre-categorization of images, get image statistics
image_statistics = []
count = 0
for listing_id in listdir(paths.RAW_IMAGES):
    listing_id_path = join(paths.RAW_IMAGES, listing_id)
    product_sum = 0
    diag_sum = 0
    width_sum = 0
    height_sum = 0
    if listing_id == ".DS_Store":
        continue
    num_images = len(listdir(listing_id_path))
    if num_images == 0:
        continue
    for img_name in listdir(join(paths.RAW_IMAGES, listing_id)):
        if not img_name.endswith('.jpg'):
            num_images -= 1
            continue
        img = Image.open(join(listing_id_path, img_name))
        width_sum += img.size[0]
        height_sum += img.size[1]
        product_sum += img.size[0] * img.size[1]
        diag_sum += math.sqrt(img.size[0]**2 + img.size[1]**2)
        count += 1
        if count % 100000 == 0:
            print "finished {} images".format(count)
    if width_sum == 0:
        continue
    image_statistics.append([listing_id, float(width_sum)/num_images,
                             float(height_sum)/num_images,  
                             float(product_sum)/num_images,
                             float(diag_sum)/num_images])

npy_file = np.array(image_statistics)
np.save(paths.IMG_STATISTICS, npy_file)
print "Shape is {}".format(npy_file.shape)

finished 100000 images
finished 200000 images
finished 300000 images
finished 400000 images
finished 500000 images
finished 600000 images
Shape is (114773, 5)


In [24]:
duplicate_images = []

In [ ]:
df = pd.read_json(paths.TRAIN_JSON)
listing_id_interest = df[['listing_id', 'interest_level']]

In [25]:
# move everything into train that is in train
for entry in listing_id_interest.iterrows():
    for img_name in listdir(join(paths.RAW_IMAGES, str(entry[1].listing_id))):
        img_path = join(paths.RAW_IMAGES, str(entry[1].listing_id), img_name)
        img_dest = join(paths.TRN_IMAGES, entry[1].interest_level)
        if img_path.endswith('.jpg'):
            try:
                shutil.move(img_path, img_dest)
            except:
                duplicate_images.append(img_path) 

In [26]:
print len(duplicate_images)

209


In [28]:
df_test = pd.read_json(paths.TEST_JSON)
test_ids = df_test['listing_id']

In [32]:
duplicate_test_images = []

In [33]:
# move everything into test which is in test
for test_id in test_ids:
    for img_name in listdir(join(paths.RAW_IMAGES, str(test_id))):
        img_path = join(paths.RAW_IMAGES, str(test_id), img_name)
        img_dest = join(paths.TEST_IMAGES)
        if img_path.endswith('.jpg'):
            try:
                shutil.move(img_path, img_dest)
            except:
                duplicate_test_images.append(img_path)

In [34]:
print len(duplicate_test_images)

906
